In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
from sklearn import preprocessing 

def train_y(y):
    y_one=np.zeros(10)
    y_one[y]=1
    return y_one
test_data=pd.read_csv('/content/drive/Shareddrives/dc競賽/test_data.csv') # DataFram:528x6001
train=pd.read_csv('/content/drive/Shareddrives/dc競賽/train.csv') # DataFram:792x6002

def data_processing(dataset):
    data_pre=dataset.iloc[:,1:6001]
    label_pre=dataset.iloc[:,6001]

    data=[]
    label=[]
    
    for i in range(data_pre.shape[0]):
        for j in range(10):
            data.append(data_pre.iloc[i][j*300:j*300+3000])
            label.append(label_pre.iloc[i])
    
    data=np.array(data)
    label=np.array(label)
      
    label_transform=np.array([train_y(label[i]) for i in range(len(label))])

    index=[i for i in range(len(data))]
    np.random.shuffle(index)
    
    trX=data[index[:]]
    trY=label_transform[index[:]]
    teX=data[index[:]]
    teY=label_transform[index[:]]        
            
    return trX,trY,teX,teY
            
trX,trY,teX,teY=data_processing(train)

def data_test_processing(dataset):
    data_pre=dataset.iloc[:,1:6001] # float64 mnist:float32

    test_data=[]
    
    for i in range(data_pre.shape[0]):
        for j in range(10):
            test_data.append(data_pre.iloc[i][j*300:j*300+3000])
    
    test_data=np.array(test_data)
    
    return test_data
            
test_data=data_test_processing(test_data)


In [ ]:
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,Input
from tensorflow.keras.layers import Conv1D,MaxPooling1D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import regularizers
import numpy as np
import os


from tensorflow.keras import backend as K

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall
 
    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

if K.image_data_format()=='channel_first':
    trX=trX.reshape(trX.shape[0],1,3000)
    teX=teX.reshape(teX.shape[0],1,3000)
    test_data=test_data.reshape(test_data.shape[0],1,3000)
    input_shape=(1,3000)
else:
    trX=trX.reshape(trX.shape[0],3000,1)
    teX=teX.reshape(teX.shape[0],3000,1)
    test_data=test_data.reshape(test_data.shape[0],3000,1)
    input_shape=(3000,1)
    
batch_size=128
num_classes=10
epochs=5000


model=Sequential()
model.add(Conv1D(16,kernel_size=3,strides=1,
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv1D(16,kernel_size=3,strides=1,
                 activation='relu',))
model.add(Conv1D(16,kernel_size=3,strides=1,
                 activation='relu',))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(32,kernel_size=3,strides=1,
                 activation='relu'))
model.add(Conv1D(32,kernel_size=3,strides=1,
                 activation='relu'))
model.add(Conv1D(32,kernel_size=3,strides=1,
                 activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64,kernel_size=3,strides=1,
                 activation='relu'))
model.add(Conv1D(64,kernel_size=3,strides=1,
                 activation='relu'))
model.add(Conv1D(64,kernel_size=3,strides=1,
                 activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes,activation='softmax',
               activity_regularizer=regularizers.l2(0.0002)))

sgd=SGD(lr=0.005,decay=1e-5)

es = tensorflow.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', mode='auto', verbose=1, patience=5, restore_best_weights=True)

model.compile(loss=tensorflow.keras.losses.categorical_crossentropy,
              optimizer=sgd,
              metrics=['accuracy',f1,recall,precision])

print(model.metrics_names)
model.summary()

In [ ]:
model.fit(trX,trY,batch_size=batch_size,epochs=1,validation_data=(teX,teY),verbose=1, callbacks=[es])
test_label=model.predict_classes(test_data)
print(test_label)

In [ ]:
import numpy as np

print(np.array(test_label).shape) 
label_process=test_label.reshape(528,10)


label=[]

for i in range(528):
    label.append([i+1,label_process[i][np.argmax(list(label_process[i]).count(x) for x in set(label_process[i]))]])

import csv
with open('test_label14.csv','w',newline='') as csvfile:
    writer=csv.writer(csvfile)
    writer.writerow(['id','label'])
    for row in label:
        writer.writerow(row)
